<a href="https://colab.research.google.com/github/trisha-agni/makemore_MLP/blob/main/makemore_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [6]:
len(words)

32033

In [7]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
print(stoi)
print(itos)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [8]:
# build the dataset
block_size = 3 # context length: num chars needed to predict next char
X, Y = [], []
for w in words[:5]:
  print(w)
  context = [0] * block_size
  for char in w + '.':
    ix = stoi[char]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [9]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [10]:
X

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1],
        [ 0,  0,  0],
        [ 0,  0, 15],
        [ 0, 15, 12],
        [15, 12,  9],
        [12,  9, 22],
        [ 9, 22,  9],
        [22,  9,  1],
        [ 0,  0,  0],
        [ 0,  0,  1],
        [ 0,  1, 22],
        [ 1, 22,  1],
        [ 0,  0,  0],
        [ 0,  0,  9],
        [ 0,  9, 19],
        [ 9, 19,  1],
        [19,  1,  2],
        [ 1,  2,  5],
        [ 2,  5, 12],
        [ 5, 12, 12],
        [12, 12,  1],
        [ 0,  0,  0],
        [ 0,  0, 19],
        [ 0, 19, 15],
        [19, 15, 16],
        [15, 16,  8],
        [16,  8,  9],
        [ 8,  9,  1]])

In [11]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [12]:
C = torch.randn((27, 2))
C

tensor([[ 0.6599, -1.1703],
        [-1.4582,  1.1701],
        [-0.6291,  0.7913],
        [-0.5775,  0.3662],
        [ 0.2600, -0.1994],
        [ 1.7383, -1.1982],
        [ 1.9286,  0.2967],
        [ 1.2202,  0.2195],
        [ 0.0141, -0.6183],
        [-0.6503, -0.2354],
        [-0.0464, -1.3993],
        [-0.0468, -0.2682],
        [ 0.0811, -0.1543],
        [-0.7913,  0.8419],
        [-1.4261,  0.2657],
        [ 0.4062,  2.3223],
        [-0.8578,  0.1309],
        [ 2.1426,  0.3087],
        [ 0.9276,  0.2278],
        [-0.8657,  0.0622],
        [-0.3555, -0.7876],
        [ 1.2819, -0.9415],
        [ 0.0642, -0.7504],
        [ 1.2434, -1.1916],
        [ 0.0118, -0.8943],
        [-0.3805,  0.4268],
        [ 0.9072,  2.1625]])

In [13]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [14]:
emb

tensor([[[ 0.6599, -1.1703],
         [ 0.6599, -1.1703],
         [ 0.6599, -1.1703]],

        [[ 0.6599, -1.1703],
         [ 0.6599, -1.1703],
         [ 1.7383, -1.1982]],

        [[ 0.6599, -1.1703],
         [ 1.7383, -1.1982],
         [-0.7913,  0.8419]],

        [[ 1.7383, -1.1982],
         [-0.7913,  0.8419],
         [-0.7913,  0.8419]],

        [[-0.7913,  0.8419],
         [-0.7913,  0.8419],
         [-1.4582,  1.1701]],

        [[ 0.6599, -1.1703],
         [ 0.6599, -1.1703],
         [ 0.6599, -1.1703]],

        [[ 0.6599, -1.1703],
         [ 0.6599, -1.1703],
         [ 0.4062,  2.3223]],

        [[ 0.6599, -1.1703],
         [ 0.4062,  2.3223],
         [ 0.0811, -0.1543]],

        [[ 0.4062,  2.3223],
         [ 0.0811, -0.1543],
         [-0.6503, -0.2354]],

        [[ 0.0811, -0.1543],
         [-0.6503, -0.2354],
         [ 0.0642, -0.7504]],

        [[-0.6503, -0.2354],
         [ 0.0642, -0.7504],
         [-0.6503, -0.2354]],

        [[ 0.0642, -0

In [15]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [16]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [17]:
h

tensor([[ 0.8246,  0.3283, -0.5424,  ...,  0.7922,  0.9861,  0.2610],
        [ 0.9848,  0.9815, -0.8515,  ...,  0.9338,  0.9408, -0.9116],
        [-0.4465, -0.8339, -0.9949,  ...,  0.7536, -0.3565,  0.7392],
        ...,
        [-0.8355, -0.7754,  0.9858,  ...,  0.9893,  0.9728,  0.8899],
        [-0.9740, -0.9935,  0.7531,  ...,  0.6228,  0.6833,  0.9912],
        [-0.9488, -0.5451, -0.9827,  ..., -0.8265, -0.7337,  1.0000]])

In [18]:
h.shape

torch.Size([32, 100])

In [19]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [20]:
logits = h @ W2 + b2

In [21]:
logits.shape

torch.Size([32, 27])

In [22]:
counts = logits.exp()

In [23]:
prob = counts / counts.sum(1, keepdims = True)

In [24]:
prob.shape

torch.Size([32, 27])

In [25]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(17.1688)